# Evalution

## Imports:

In [12]:
import pandas as p
from pathlib import Path
import re

## Get Models:

In [ ]:
modelsPath= Path('../Models/Weights/')
+ list(modelsPath.glob('*.pkl'))

In [ ]:
def evaluate_pytorch_models(models_path:Path):
    for i in list(modelsPath.glob('*.pth')):
        # extract model name from file name
        #eval(model_name)().load_state_dict(torch.load(i))
        # perform evaluation......
        print(i)

def evaluate_pkl_models(models_path:Path):
    for i in list(modelsPath.glob('*.pkl')):
        # load pkl file
        # perfom evaluation....
        print(i)
def evaluate_models(models_path:Path):
    get_pytorch_models(models_path)
    get_pkl_models(modelsPath)
        
get_model_weights(modelsPath)

..\Models\Weights\model1.pth
..\Models\Weights\model2.pkl
